# Import data

In [1]:
import pandas as pd

In [2]:
TRAIN = pd.read_csv('TRAIN_FINAL_NO_ITEMS.csv')
TRAIN.shape

(125497040, 104)

In [3]:
TARGET = pd.read_csv('TRAIN_TARGET.csv')
TARGET.shape

(125497040, 1)

In [4]:
TEST = pd.read_csv('TEST_FINAL_NO_ITEMS.csv')
TEST.shape

(3370464, 104)

# Build model

In [5]:
import tensorflow as tf

/home/carleton_coggins/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
columns = TRAIN.columns.values
feature_columns = []
for i in columns:
    feat = tf.feature_column.numeric_column(i)
    feature_columns.append(feat)

In [108]:
data_size = int(TARGET.shape[0] * 0.000008)
C = int(TARGET.shape[0] - (data_size*0.2))
A = int(TARGET.shape[0] - data_size - 1)
B = C - 1

x_train = TRAIN[A:B]
y_train = TARGET[A:B]
x_eval = TRAIN[C:]
y_eval = TARGET[C:]

learning_rate = 0.01
dropout = 0.85
batch_size = 2
epochs = 40
steps = int(len(x_train) / batch_size * epochs) # data size / batch_size * epochs

print("Data size: ", data_size, "\nTrain Size: ", len(x_train), "\nEval Size: ", len(x_eval), "\nSteps: ", steps)

Data size:  1003 
Train Size:  802 
Eval Size:  201 
Steps:  16040


In [109]:
estimator = tf.estimator.DNNRegressor(hidden_units=[100,50,20,10],
                                      feature_columns=feature_columns,
                                      model_dir="model_dir/",
                                      label_dimension=1,
                                      optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate),
                                      dropout=dropout,
                                      # activation_fn=tf.nn.relu
                                      
                                     )

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model_dir/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1a6cb31b38>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [110]:
input_fn = tf.estimator.inputs.pandas_input_fn(
    x_train, y_train, batch_size=batch_size, num_epochs=None, shuffle=True)

train_input_fn = tf.estimator.inputs.pandas_input_fn(
    x_train, y_train, batch_size=batch_size, num_epochs=epochs, shuffle=False)

eval_input_fn = tf.estimator.inputs.pandas_input_fn(
    x_eval, y_eval, batch_size=batch_size, num_epochs=epochs, shuffle=False)

In [111]:
# Documentation seems to suggest that the input_fn should go to estimator.train(), 
# with no epochs, and the data shuffled...
# but for some reason we pass train_input_fn in to estimator.train()

estimator.train(input_fn=input_fn, steps=steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into model_dir/model.ckpt.
INFO:tensorflow:loss = 1.6288352e+18, step = 1
INFO:tensorflow:global_step/sec: 100.614
INFO:tensorflow:loss = 4085112000000000.0, step = 101 (1.008 sec)
INFO:tensorflow:global_step/sec: 133.647
INFO:tensorflow:loss = 40546.668, step = 201 (0.754 sec)
INFO:tensorflow:global_step/sec: 133.307
INFO:tensorflow:loss = 92654180000000.0, step = 301 (0.744 sec)
INFO:tensorflow:global_step/sec: 135.287
INFO:tensorflow:loss = 53.779217, step = 401 (0.739 sec)
INFO:tensorflow:global_step/sec: 136.076
INFO:tensorflow:loss = 388.8812, step = 501 (0.735 sec)
INFO:tensorflow:global_step/sec: 135.331
INFO:tensorflow:loss = 210350840000000.0, step = 601 (0.745 sec)
INFO:tensorflow:global_step/sec: 135.

INFO:tensorflow:global_step/sec: 140.636
INFO:tensorflow:loss = 220444.12, step = 8001 (0.706 sec)
INFO:tensorflow:global_step/sec: 137.289
INFO:tensorflow:loss = 3515802200000.0, step = 8101 (0.730 sec)
INFO:tensorflow:global_step/sec: 134.787
INFO:tensorflow:loss = 5.475316, step = 8201 (0.740 sec)
INFO:tensorflow:global_step/sec: 137.962
INFO:tensorflow:loss = 68.61068, step = 8301 (0.727 sec)
INFO:tensorflow:global_step/sec: 138.705
INFO:tensorflow:loss = 37.95049, step = 8401 (0.721 sec)
INFO:tensorflow:global_step/sec: 135.871
INFO:tensorflow:loss = 14115.208, step = 8501 (0.734 sec)
INFO:tensorflow:global_step/sec: 134.788
INFO:tensorflow:loss = 91.38566, step = 8601 (0.736 sec)
INFO:tensorflow:global_step/sec: 137.826
INFO:tensorflow:loss = 218081.56, step = 8701 (0.732 sec)
INFO:tensorflow:global_step/sec: 138.775
INFO:tensorflow:loss = 17.692385, step = 8801 (0.727 sec)
INFO:tensorflow:global_step/sec: 137.586
INFO:tensorflow:loss = 38.127426, step = 8901 (0.722 sec)
INFO:ten

In [112]:
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-01-00:46:43
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_dir/model.ckpt-16040
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-01-00:48:37
INFO:tensorflow:Saving dict for global step 16040: average_loss = 742.5275, global_step = 16040, loss = 1485.055
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-01-00:48:38
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_dir/model.ckpt-16040
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-01-00:49:07
INFO:tensorflow:Saving dict for global step 16040: average_loss = 20314.953, global_step = 16040, loss = 40629.906
train metric

# Test model

In [113]:
pred_input_fn = tf.estimator.inputs.pandas_input_fn(
    TEST[:25], shuffle=False)

In [114]:
predictions = list(estimator.predict(input_fn=pred_input_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_dir/model.ckpt-16040
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [115]:
predict_df = pd.DataFrame([i['predictions'][0] for i in predictions])

In [116]:
predict_df

,0
0,-1.341448
1,-1.341448
2,-1.341448
3,-1.341448
4,-1.341448
5,-1.341448
6,-1.341448
7,-1.341448
8,-1.341448
9,-1.341448


In [ ]:
'''
FOR RECORDING PREDICTIONS

predictions = list(classifier.predict(input_fn=predict_input_fn))
  predicted_classes = [p["classes"] for p in predictions]

  print(
      "New Samples, Class Predictions:    {}\n"
      .format(predicted_classes))
'''